In [1]:
import torch
import os
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

model_name = 'int_2'
model = torch.load(os.path.join('models',model_name,f'{model_name}.pth'))
model.eval()

Device: cuda:1
Current cuda device: 0
Count of using GPUs: 2


UNet3d(
  (conv): DoubleConv(
    (double_conv): Sequential(
      (0): Conv3d(3, 24, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (1): GroupNorm(8, 24, eps=1e-05, affine=True)
      (2): ReLU(inplace=True)
      (3): Conv3d(24, 24, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (4): GroupNorm(8, 24, eps=1e-05, affine=True)
      (5): ReLU(inplace=True)
    )
  )
  (enc1): Down(
    (encoder): Sequential(
      (0): MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv3d(24, 48, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
          (1): GroupNorm(8, 48, eps=1e-05, affine=True)
          (2): ReLU(inplace=True)
          (3): Conv3d(48, 48, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
          (4): GroupNorm(8, 48, eps=1e-05, affine=True)
          (5): ReLU(inplace=True)
        )
      )
    )
  )
  (

In [2]:
import json
with open('models/model_subscriptions.json','r') as f:
    models_info = json.load(f)
model_info = next((model for model in models_info if model['model_name'] == model_name), None)
model_info

{'model_name': 'int_2',
 'depth / img_size / in_channel / n_channel': [78, 120, 3, 24],
 'used_channel': ['-t1n.nii.gz', '-t1c.nii.gz', '-t2f.nii.gz'],
 'val score(dice/jaccard)': [0.849, 0.754],
 'batch/total epoch/best epoch': [4, 20, 16],
 'resize_info': [0, 155, 2],
 'run time(m)': 147}

In [3]:
from utils import get_dataloader
from dataset import BratsDataset

test_dataloader = get_dataloader(dataset=BratsDataset, phase="test", resize_info=model_info['resize_info'], img_width=model_info['depth / img_size / in_channel / n_channel'][1], data_type=model_info['used_channel'])

In [4]:
from tqdm import tqdm
tp = 0
fp = 0
tn = 0
fn = 0
total_predictions = []
counter = 0  # Counter to keep track of the number of entries processed

with torch.no_grad():  # Disable gradient calculations to save memory
    for data in tqdm(test_dataloader):
        
        images, targets = data['image'], data['mask']
        images = images.to(device)
        targets = targets.to(device)

        logits = model(images)
        probabilities = torch.sigmoid(logits)
        predictions = (probabilities >= 0.33).float()
        total_predictions.append(predictions)
        # Compute binary segmentation metrics
        tp += torch.sum((predictions == 1) & (targets == 1)).item()
        fp += torch.sum((predictions == 1) & (targets == 0)).item()
        tn += torch.sum((predictions == 0) & (targets == 0)).item()
        fn += torch.sum((predictions == 0) & (targets == 1)).item()

        counter += 1

        # Free memory by clearing intermediate variables
        del images, targets, logits, probabilities, predictions
        torch.cuda.empty_cache()


  0%|          | 0/32 [00:05<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 824.00 MiB (GPU 0; 11.87 GiB total capacity; 1.39 GiB already allocated; 770.19 MiB free; 1.54 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
print(f"True positives : {tp}")
print(f"False positives : {fp}")
print(f"True Negatives : {tn}")
print(f"False Negatives : {fn}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_confusion_matrix(tp, fp, tn, fn):
    # Create confusion matrix array
    confusion_matrix = np.array([[tn, fp], [fn, tp]])

    # Set up labels for matrix
    labels = ['True ', 'False ']

    # Create color map
    cmap = plt.cm.Blues

    # Plot confusion matrix
    plt.imshow(confusion_matrix, interpolation='nearest', cmap=cmap)
    plt.title('Confusion Matrix')
    plt.colorbar()

    # Add labels to matrix cells
    thresh = confusion_matrix.max() / 2.
    for i, j in np.ndindex(confusion_matrix.shape):
        plt.text(j, i, format(confusion_matrix[i, j], 'd'), horizontalalignment='center', color='white' if confusion_matrix[i, j] > thresh else 'black')

    # Set tick labels
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)

    # Set axis labels
    plt.xlabel('Predicted label')
    plt.ylabel('True label')

    # Show plot
    plt.show()

plot_confusion_matrix(tp, fp, tn, fn)


In [ ]:
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2 * (precision * recall) / (precision + recall)

In [ ]:
print(f"Accuracy : {accuracy*100}")
print(f"Precision : {precision*100}")
print(f"Recall : {recall*100}")
print(f"F1 Score : {f1_score*100}")

In [ ]:
import torch
import gc 
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm
gc.collect()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

predictions = []
total_targets = []
# Counter to keep track of the number of entries processed
counter = 0  

with torch.no_grad():
    for data in tqdm(test_dataloader):
        if counter >= 5:
            break  # Stop processing entries if the desired number is reached

        images, targets = data['image'], data['mask']
        images = images.to(device)
        targets = targets.to(device)

        logits = model(images)
        probabilities = torch.sigmoid(logits)
        prediction = (probabilities >= 0.33).float()

        prediction =  prediction.cpu()
        targets = targets.cpu()
        total_targets.append(targets)
        predictions.append(prediction)

        model.zero_grad()
        del images, targets, logits, probabilities, prediction
        torch.cuda.empty_cache()

        counter += 1

In [ ]:
len(predictions)

In [ ]:
# Compute confusion matrix
y_true = np.concatenate(total_targets)
y_pred = np.concatenate(predictions)

y_true.shape, y_pred.shape

In [ ]:
from utils import dice_coef_metric_per_classes, jaccard_coef_metric_per_classes
def compute_scores_per_classes(model,          # nodel which is UNeT3D 
                               dataloader,     # tuple consisting of ( id , image tensor , mask tensor )
                               classes):       # classes : WT , TC , ET 
    """
    Compute Dice and Jaccard coefficients for each class.
    Params:
        model: neural net for make predictions.
        dataloader: dataset object to load data from.
        classes: list with classes.
        Returns: dictionaries with dice and jaccard coefficients for each class for each slice.
    """
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    dice_scores_per_classes = {key: list() for key in classes}
    iou_scores_per_classes = {key: list() for key in classes}

    with torch.no_grad():
        for i, data in enumerate(dataloader):
            imgs, targets = data['image'], data['mask']
            imgs, targets = imgs.to(device), targets.to(device)
            logits = model(imgs)
            logits = logits.detach().cpu().numpy()
            targets = targets.detach().cpu().numpy()
            
            # Now finding the overlap between the raw prediction i.e. logit & the mask i.e. target & finding the dice & iou scores 
            dice_scores = dice_coef_metric_per_classes(logits, targets)
            iou_scores = jaccard_coef_metric_per_classes(logits, targets)

            # storing both dice & iou scores in the list declared 
            for key in dice_scores.keys():
                dice_scores_per_classes[key].extend(dice_scores[key])

            for key in iou_scores.keys():
                iou_scores_per_classes[key].extend(iou_scores[key])

    return dice_scores_per_classes, iou_scores_per_classes 

In [ ]:
dice_scores_per_classes, iou_scores_per_classes = compute_scores_per_classes(
    model, test_dataloader, ['WT', 'TC', 'ET']
    )

In [ ]:
import pandas as pd
dice_df = pd.DataFrame(dice_scores_per_classes)
dice_df.columns = ['WT dice', 'TC dice', 'ET dice']

iou_df = pd.DataFrame(iou_scores_per_classes)
iou_df.columns = ['WT jaccard', 'TC jaccard', 'ET jaccard']
# CONCAT BOTH THE COLUMNS ALONG AXIS 1 & SORT THE TWO 
val_metics_df = pd.concat([dice_df, iou_df], axis=1, sort=True)
val_metics_df = val_metics_df.loc[:, ['WT dice', 'WT jaccard', 
                                      'TC dice', 'TC jaccard', 
                                      'ET dice', 'ET jaccard']]
val_metics_df.sample(5)

In [ ]:
[i for i in val_metics_df.std()]

In [ ]:
val_metics_df.mean()

In [ ]:
import seaborn as sns

colors = ['#264653', '#2a9d8f', '#8ab17d', '#e9c46a', '#f4a261', '#e76f51']
palette = sns.color_palette(colors, 6)

fig, ax = plt.subplots(figsize=(12, 6))
sns.barplot(x=val_metics_df.mean().index, y=val_metics_df.mean(), palette=palette, ax=ax)
ax.set_xticklabels(val_metics_df.columns, fontsize=14, rotation=15)
ax.set_title("Dice and Jaccard Coefficients from Test", fontsize=20)

for idx, p in enumerate(ax.patches):
        percentage = '{:.1f}%'.format(100 * val_metics_df.mean().values[idx])
        x = p.get_x() + p.get_width() / 2 - 0.15
        y = p.get_y() + p.get_height()
        ax.annotate(percentage, (x, y), fontsize=15, fontweight="bold")

fig.savefig("result1.png", format="png",  pad_inches=0.2, transparent=False, bbox_inches='tight')
fig.savefig("result1.svg", format="svg",  pad_inches=0.2, transparent=False, bbox_inches='tight')